In [1]:
%matplotlib inline
import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import psycopg2
from dotenv import load_dotenv

In [2]:
#https://www.youtube.com/watch?v=MEEVcpnfElY
leituras_dique6c = pd.read_csv('../arquivos/leituras_dique6c.csv')
leituras_bvsa = pd.read_csv('../arquivos/leituras_bvsa.csv')

In [3]:
loc_bvsa = pd.read_csv('../arquivos/loc_bvsa.csv', sep=';', decimal=',')
loc_dique6c = pd.read_csv('../arquivos/loc_dique6c.csv', sep=';', decimal=',')

In [4]:
loc_sensores = pd.read_excel('../arquivos/LOCALIZAÇÃO_SENSORES.xlsx', engine='openpyxl', header=2)

In [5]:
leituras_dique6c['area'] = 'BVSA'
leituras_bvsa['area'] = 'Dique 6C'

In [6]:
leituras = pd.concat([leituras_bvsa, leituras_dique6c])
leituras = leituras.rename(columns={"Sensor":"sensor", "Data":"data", "Valor":"valor", "Unidade":"unidade", "Direcao_Saida":"direcao_saida", "Estado":"estado"})
leituras['direcao_saida'] = leituras['direcao_saida'].replace('Cota NA', 'Cota do NA')

In [7]:
leituras['sensor'] = leituras['sensor']\
    .apply(lambda \
        x: x.replace("(I)", '')\
            .replace("(S)", '')\
                .replace("‐", '-')\
                    .replace("-ER", '')\
                        .replace("-TC", '')\
                            .strip())

In [8]:
leituras['direcao_saida'].value_counts()

Cota Piezométrica                             19204
Carga Piezométrica                            19204
Vazão                                         10554
Cota do NA                                     6227
Recalque (Z)                                   1007
Deslocamento Longitudinal (X)                  1007
Deslocamento Transversal (Y)                   1007
Recalque                                        819
Deslocamento da Estaca Dir(-)/Esq(+)            819
Deslocamento do Afastamento Mont(-)/Jus(+)      819
Leitura                                          32
Name: direcao_saida, dtype: int64

In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

leituras['id_unidade'] = le.fit_transform(leituras['unidade']) +1
leituras['id_direcao_saida'] = le.fit_transform(leituras['direcao_saida']) +1
leituras['id_estado'] = le.fit_transform(leituras['estado']) +1
leituras['id_area'] = le.fit_transform(leituras['area']) + 1

leituras = leituras.reset_index(drop=True)

In [10]:
leituras.groupby(by=['unidade', 'direcao_saida', 'estado', 'area']).agg(pd.Series.nunique)[['sensor']].sort_values(by=['sensor'], ascending=False)

sensor
unidade direcao_saida                              estado  area            
m.c.a   Carga Piezométrica                         normal  BVSA          36
m       Cota Piezométrica                          normal  BVSA          36
        Cota do NA                                 normal  BVSA          35
m.c.a   Carga Piezométrica                         normal  Dique 6C      29
m       Cota Piezométrica                          normal  Dique 6C      29
        Deslocamento Longitudinal (X)              normal  BVSA          18
        Deslocamento da Estaca Dir(-)/Esq(+)       normal  BVSA          18
        Deslocamento do Afastamento Mont(-)/Jus(+) normal  BVSA          18
        Recalque                                   normal  BVSA          18
        Recalque (Z)                               normal  Dique 6C      18
        Deslocamento Longitudinal (X)              normal  Dique 6C      18
        Deslocamento Transversal (Y)               normal  BVSA          18
                                                           Dique 6C      18
        Recalque (Z)                               normal  BVSA          18
        Recalque                                   normal  Dique 6C       8
        Deslocamento da Estaca Dir(-)/Esq(+)       normal  Dique 6C       8
        Deslocamento do Afastamento Mont(-)/Jus(+) normal  Dique 6C       8
l/s     Vazão                                      normal  BVSA           7
l/min   Vazão                                      normal  BVSA           7
m       Recalque                                   atencao BVSA           6
m3/s    Vazão                                      normal  BVSA           6
                                                           Dique 6C       4
m       Deslocamento do Afastamento Mont(-)/Jus(+) atencao BVSA           4
l/min   Vazão                                      normal  Dique 6C       4
l/s     Vazão                                      normal  Dique 6C       4
m       Deslocamento do Afastamento Mont(-)/Jus(+) atencao Dique 6C       3
        Cota do NA                                 normal  Dique 6C       2
        Recalque                                   atencao Dique 6C       1
        Deslocamento da Estaca Dir(-)/Esq(+)       atencao BVSA           1
m.c.a   Carga Piezométrica                         alerta  Dique 6C       1
                                                   atencao BVSA           1
                                                           Dique 6C       1
m       Cota Piezométrica                          atencao Dique 6C       1
                                                           BVSA           1
m3/min  Vazão                                      normal  BVSA           1
m       Cota Piezométrica                          alerta  Dique 6C       1
seg     Leitura                                    normal  BVSA           1

In [11]:
unidade = leituras[['id_unidade', 'unidade']].sort_values(by=['id_unidade']).drop_duplicates()
direcao_saida = leituras[['id_direcao_saida', 'direcao_saida']].sort_values(by=['id_direcao_saida']).drop_duplicates()
estado = leituras[['id_estado', 'estado']].sort_values(by=['id_estado']).drop_duplicates()
area = leituras[['id_area', 'area']].sort_values(by=['id_area']).drop_duplicates()

In [12]:
#Criação da tabela de sensores

In [13]:
tabela_sensores = leituras[['sensor']].drop_duplicates().sort_values(by=['sensor']).reset_index(drop=True)
tabela_sensores.index = tabela_sensores.index+1

In [14]:
tabela_sensores.to_excel('../tabelas/sensores.xlsx')

In [15]:
tabela_sensores

,sensor
1,MNA-AEF1-01
2,MNA-AEF1-02
3,MNA-BM-BV-51
4,MNA-BM-BV-81
5,MNA-SM-7500A
...,...
145,PZ-JF1-02
146,PZ-JF1-03
147,PZ-JF1-04
148,PZ-JF1-05


In [16]:
dict_sensores = tabela_sensores.reset_index(names=['id'])[['sensor','id']].to_dict(orient = 'tight')
dict_sensores = dict(dict_sensores['data']);

In [17]:
leituras['id_sensor'] = leituras['sensor'].map(dict_sensores)

In [18]:
leituras[leituras['id_sensor'].isna()]

,sensor,data,valor,unidade,direcao_saida,estado,area,id_unidade,id_direcao_saida,id_estado,id_area,id_sensor


In [19]:
leituras['id_sensor'] = leituras['id_sensor'].apply(lambda x: int(x))

In [20]:
leituras = leituras[['id_sensor','id_unidade', 'id_direcao_saida', 'id_estado', 'id_area', 'sensor', 'data', 'valor', 'unidade', 'direcao_saida', 'estado', 'area']]

In [21]:
leituras = leituras.drop_duplicates()

In [22]:
#Tratamento dos dados de localização
import geopandas as gpd

In [23]:
localizacao = gpd.GeoDataFrame(pd.concat([loc_bvsa, loc_dique6c]))
localizacao = localizacao.rename(columns={"Sensor":"sensor", "Latitude":"latitude", "Longitude":"longitude", "Altura":"altura"})

In [24]:
localizacao['sensor'] = localizacao['sensor'].apply(lambda x: x.replace("(I)", '').replace("(S)", '').replace("‐", '-').strip())

In [25]:
localizacao['id_sensor'] = localizacao['sensor'].map(dict_sensores)

In [26]:
localizacao[localizacao['id_sensor'].isnull()]['sensor'].unique()

array(['RN-BM-BV-01', 'RN-BM-BV-02', 'RN-BM-BV-03', 'RN-BM-BV-04',
       'IN-BM-BV-11', 'IN-BM-BV-12', 'MR-BM-BV-11', 'MR-BM-BV-31',
       'PZ-D06C-18', 'RN-D06C-01', 'RN-D06C-02', 'RN-D06C-03',
       'RN-D06C-04', 'RM-D06C-01', 'RM-D06C-02', 'RM-D06C-03',
       'RM-D06C-04'], dtype=object)

In [27]:
localizacao = localizacao.reset_index(drop=True)
localizacao.index = localizacao.index + 1
localizacao

,sensor,N,E,latitude,longitude,altura,id_sensor
1,PZ-BM-BV-11,9.652626e+09,412507056.0,-3.142831,-51.787880,74.48,91.0
2,PZ-BM-BV-12,9.652626e+09,412507056.0,-3.142831,-51.787880,59.15,92.0
3,PZ-BM-BV-13,9.652636e+09,412475409.0,-3.142733,-51.788165,69.96,93.0
4,PZ-BM-BV-14,9.652636e+09,412475409.0,-3.142733,-51.788165,54.44,94.0
5,PZ-BM-BV-15,9.652622e+09,412336622.0,-3.142866,-51.789414,65.00,95.0
...,...,...,...,...,...,...,...
117,MV-D06C-04,9.652152e+09,413817837.0,-3.147122,-51.776087,65.00,60.0
118,RM-D06C-01,9.651928e+09,413655231.0,-3.149147,-51.777552,100.00,NaN
119,RM-D06C-02,9.651928e+09,413655231.0,-3.149147,-51.777552,100.00,NaN
120,RM-D06C-03,9.651928e+09,413655231.0,-3.149147,-51.777552,100.00,NaN


In [28]:
x_latitude = localizacao['latitude']
y_longitude = localizacao['longitude']
z_altura = localizacao['altura']

In [29]:
localizacao['geom'] = gpd.GeoSeries.from_xy(x =y_longitude, y=x_latitude, z=z_altura, crs='EPSG:4326')

In [30]:
localizacao = localizacao[['id_sensor', 'sensor', 'latitude', 'longitude', 'altura', 'geom']]

In [31]:
localizacao.head()

,id_sensor,sensor,latitude,longitude,altura,geom
1,91.0,PZ-BM-BV-11,-3.142831,-51.787880,74.48,POINT Z (-51.78788 -3.14283 74.48000)
2,92.0,PZ-BM-BV-12,-3.142831,-51.787880,59.15,POINT Z (-51.78788 -3.14283 59.15000)
3,93.0,PZ-BM-BV-13,-3.142733,-51.788165,69.96,POINT Z (-51.78816 -3.14273 69.96000)
4,94.0,PZ-BM-BV-14,-3.142733,-51.788165,54.44,POINT Z (-51.78816 -3.14273 54.44000)
5,95.0,PZ-BM-BV-15,-3.142866,-51.789414,65.00,POINT Z (-51.78941 -3.14287 65.00000)


In [32]:
from libary.Database import Database

In [33]:
database = Database(schema='pesquisa', table='view_leituras_sensores', path='../.env', ambiente = 1)

TypeError: __init__() got an unexpected keyword argument 'dotenv_path'

In [ ]:
localizacao = localizacao.set_geometry('geom')

In [ ]:
localizacao_ = localizacao[['id_sensor', 'sensor', 'latitude', 'longitude', 'altura']]

In [ ]:
leituras = leituras.rename(columns={'index': 'id'})
localizacao = localizacao.rename(columns={'index': 'id'})

In [ ]:
from libary.Database import Database

In [ ]:
database = Database(schema='pesquisa', table='area', ambiente = 0, dotenv_path='../.env')
database.truncate_table()
print('truncate ok')
database.to_table(df=area, schema='pequisa', table='area')

truncate ok
Falha ao importar as variaveis de ambiente.
oi


In [ ]:
ysrewgersdergsdt

NameError: name 'ysrewgersdergsdt' is not defined

In [ ]:
database.truncate_table(schema='pequisa', table='sensores')
database.to_table(df=tabela_sensores, schema='pequisa', table='sensores')

database.truncate_table(schema='pequisa', table='unidade')
database.to_table(df=unidade, schema='pequisa', table='unidade')

database.truncate_table(schema='pequisa', table='direcao_saida')
database.to_table(df=direcao_saida, schema='pequisa', table='direcao_saida')

database.truncate_table(schema='pequisa', table='area')
database.to_table(df=area, schema='pequisa', table='area')

database.truncate_table(schema='pequisa', table='estado')
database.to_table(df=estado, schema='pequisa', table='estado')

database.truncate_table(schema='pequisa', table='leituras')
database.to_table(df=leituras, schema='pequisa', table='leituras')

database.truncate_table(schema='pequisa', table='localizacao')
database.to_table(df=localizacao, schema='pequisa', table='localizacao', geom=True)

In [ ]:
asfsdasdfvggfse

In [ ]:
tabela_sensores.to_sql('sensores', schema='pesquisa', con=conn, if_exists='replace', index=True)
unidade.to_sql('unidade', schema='pesquisa', con=conn, if_exists='replace', index=False)
direcao_saida.to_sql('direcao_saida', schema='pesquisa', con=conn, if_exists='replace', index=False)
area.to_sql('area', schema='pesquisa', con=conn, if_exists='replace', index=False)
estado.to_sql('estado', schema='pesquisa', con=conn, if_exists='replace', index=False)
leituras.to_sql('leituras', schema='pesquisa', con=conn, if_exists='replace', index=True)
localizacao.to_postgis('localizacao', schema='pesquisa', con=conn, if_exists='replace', index=True)

In [ ]:
tabela_sensores.to_excel('../tabelas/sensores.xlsx')
unidades.to_excel('../tabelas/unidades.xlsx')
direcao_saida.to_excel('../tabelas/direcao_saida.xlsx')
leituras.to_excel('../tabelas/leituras.xlsx')
localizacao.to_excel('../tabelas/localizacao.xlsx')